# TESS Early Release TICA products custom extraction

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
import lightkurve as lk
from astropy.time import Time
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## TESS Lightcurves

In [ ]:
btjd_now = Time.now().jd - 2457000.0
btjd_now

## Early Release data products

TICA now offers [early release data products](https://heasarc.gsfc.nasa.gov/docs/tess/a-beta-trial-of-quick-look-tess-orbit-data-early-release-of-sector-35-orbit-1-calibrated-full-frame-images.html) through TICA full frame images.

There are two ways to try to access TICA:

- With tess-cloud, vis-a-vis the demo on [this GitHub Issue](https://github.com/lightkurve/lightkurve/issues/993)
- Download the full FFI frames yourself from the [TICA curl scripts](https://archive.stsci.edu/hlsp/tica)

Both of these strategies are not well-proven!  Let's use `tess-point` to get the coordinates.

### Luhman 16

`$ python -m tess_stars2px -c 162.303282427 -53.317573814`

```bash
# TIC     |   RA      |   Dec     | EclipticLong | EclipticLat | Sector | Camera | Ccd | ColPix | RowPix | EdgeWarn
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 10 | 2 | 1 | 1766.327733 |    7.550842 | 0
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 36 | 3 | 1 |  541.096313 |  156.639815 | 0
000000000 | 162.303282 | -53.317574 | 194.986936 | -53.902482 | 37 | 3 | 2 | 1054.536935 |   55.021891 | 0
```

In [ ]:
from astropy.io import fits
import glob
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np
import tqdm

In [ ]:
object_name = 'Luhman 16'
sr = lk.search_tesscut(object_name)
tpf = sr.download(cutout_size=(41, 15))

In [ ]:
tpf

Ok, so we want camera 4, ccd 1.

In [ ]:
tica_fns = glob.glob('../../data/TESS/TICA/s0036/cam3-ccd1/*.fits')

Get a preliminary image to set the mask

In [ ]:
hdus = fits.open(tica_fns[123])

hdr = hdus[0].header

In [ ]:
time_btjd = hdr['MIDTJD']

In [ ]:
wcs = WCS(hdr)

In [ ]:
coord = SkyCoord(lc.RA*u.deg, lc.Dec*u.deg)

yc, xc = wcs.world_to_pixel(coord)
xc, yc = int(xc), int(yc)
yc, xc

Wow, really close to what TESS-point said!  Now stack all the data into a datacube:

In [ ]:
dw = 17

In [ ]:
times = []
data = []

In [ ]:
for i, fn in tqdm.tqdm(enumerate(tica_fns)):
    hdus = fits.open(fn)
    hdr = hdus[0].header
    times.append(hdr['MIDTJD'])
    
    img = hdus[0].data
    subimg = img[xc-dw:xc+dw, yc-dw:yc+dw]
    data.append(subimg)
    hdus.close()

In [ ]:
datacube = np.array(data)
times = np.array(times)
indices = times.argsort()
times = times[indices]
datacube = datacube[indices, :, :]

In [ ]:
median_frame = np.median(datacube, axis=0)

In [ ]:
lo, mediumlo, mid, hi = np.nanpercentile(median_frame, (1, 15, 50, 98))

Ok, good spot-check, we are pointed at the right thing!

In [ ]:
vec = np.arange(34)
xx, yy = np.meshgrid(vec, vec)
dist = np.hypot(xx-17, yy-17)

In [ ]:
aperture_mask = (median_frame > hi) & (dist < 5)

In [ ]:
bkg_mask = (median_frame < mediumlo) & (dist > 9)

In [ ]:
n_ap = np.sum(aperture_mask)

In [ ]:
n_bkg = np.sum(bkg_mask)

In [ ]:
flux = np.sum(datacube*aperture_mask, axis=(1,2)) - np.sum(datacube*bkg_mask, axis=(1,2))/n_bkg*n_ap

In [ ]:
lc_tica = lk.LightCurve(time=times, flux=flux).normalize().remove_outliers()

Neat! We achieved our goal!  We see that we sampled at least a high and low phase, so that's a win.